# Evaluate testcases on models 

In [3]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import json
from sklearn.metrics import precision_recall_fscore_support
import warnings

# Ignore all warnings
warnings.simplefilter("ignore")

In [4]:
import json
from checklist.expect import Expect
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper
from allennlp_models.pretrained import load_predictor

import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\romer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
bert = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")


error loading _jsonnet (this is expected on Windows), treating C:\Users\romer\AppData\Local\Temp\tmpwn_7krqf\config.json as plain json
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
bilstm = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

error loading _jsonnet (this is expected on Windows), treating C:\Users\romer\AppData\Local\Temp\tmpnu48a7ow\config.json as plain json


In [36]:
import json

def load_testset(path):
    with open(path, 'r') as file:
        return json.load(file)

In [37]:
# Loading all the datasets
passive_voice = load_testset('test_instances/passive_voice.json')
robustness = load_testset('test_instances/robustness.json')
switching = load_testset('test_instances/switching.json')
caused_motion = load_testset('test_instances/caused_motion.json')

## Evaluation capability of Passive Voice

In [41]:
def evaluate_voice(models, data):
    for model_name in models: 
        if model_name == "BERT":
            model = bert
        if model_name == "BiLSTM":
            model = bilstm
            
        print(f"Evaluation of capability: passive voice, using model: {model_name}.\n")
        
        pred_labels_passive = []
        true_labels_passive = []
        pred_labels_active = []
        true_labels_active = []
        total, failures = 0, 0
        failed_sentences = []
        passive_fails, active_fails = [], []
        
        for sentence in data:
            passive = sentence['test_case'][0]
            active = sentence['test_case'][1]
            true_passive_label = sentence['label']
            true_active_label = sentence['label']

            pred_passive = model.predict(passive)
            pred_active = model.predict(active)

            pred_passive_label = pred_passive['verbs'][1]['tags'][1]
            pred_active_label = pred_active['verbs'][0]['tags'][4]

            total += 2

            if pred_passive_label != true_passive_label:
                failures += 1
                failed_sentences.append(["P", passive, true_passive_label, pred_passive_label])
            if pred_active_label != true_active_label:
                failures += 1
                failed_sentences.append(["A", active, true_active_label, pred_active_label])

            pred_labels_passive.append(pred_passive_label)
            true_labels_passive.append(true_passive_label)
            pred_labels_active.append(pred_active_label)
            true_labels_active.append(true_active_label)
        
        percentage = (failures/total)*100
        for sentence in failed_sentences:
            if sentence[0] == "P":
                passive_fails.append(sentence)
            else:
                active_fails.append(sentence)
        
        print(f"Number of test cases: {total}.")
        print(f"Failures: {failures} ({percentage:.3f}%).\n")
        # Evaluate performance on passive voice test cases
        if len(passive_fails) > 0:
            print(f"--- Passive sentences test cases ---")
            print(f"Examples of failures")
            if len(passive_fails) < 4:
                for i in range(0,len(passive_fails)):
                    print(f"Sentence: {passive_fails[i][1]}")
                    print(f"True labels: {passive_fails[i][2]}")
                    print(f"Predicted labels: {passive_fails[i][3]}\n")
            else:
                for i in range(0,4):
                    print(f"Sentence: {passive_fails[i][1]}")
                    print(f"True labels: {passive_fails[i][2]}")
                    print(f"Predicted labels: {passive_fails[i][3]}\n")


        # Evaluate performance on active voice test cases
        if len(active_fails) > 0:
            print(f"--- Passive sentences test cases ---")
            print(f"Examples of failures")
            if len(active_fails) < 4:
                for i in range(0,len(active_fails)):
                    print(f"Sentence: {active_fails[i][1]}")
                    print(f"True labels: {active_fails[i][2]}")
                    print(f"Predicted labels: {active_fails[i][3]}\n")
            else:
                for i in range(0,4):
                    print(f"Sentence: {active_fails[i][1]}")
                    print(f"True labels: {active_fails[i][2]}")
                    print(f"Predicted labels: {active_fails[i][3]}\n")
    print("\n")

In [42]:
evaluate_voice(["BERT", "BiLSTM"], passive_voice)

Evaluation of capability: passive voice, using model: BERT.

Number of test cases: 384.
Failures: 2 (0.521%).

--- Passive sentences test cases ---
Examples of failures
Sentence: The writer wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The author wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARGM-LOC

Evaluation of capability: passive voice, using model: BiLSTM.

Number of test cases: 384.
Failures: 4 (1.042%).

--- Passive sentences test cases ---
Examples of failures
Sentence: The writer wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The author wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The novelist wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The creator wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2





## Evaluation capability of Robustness

In [43]:
def evaluate_robustness(models, data):
    for model_name in models: 
        if model_name == "BERT":
            model = bert
        if model_name == "BiLSTM":
            model = bilstm
        
        print(f"Evaluation of capability: robustness, using model: {model_name}.\n")
        
        pred_labels_correct = []
        true_labels_correct = []
        pred_labels_typos = []
        true_labels_typos = []
        failed_sentences = []
        total, failures = 0, 0
        
        correct_fails, typos_fails = [], []
        
        for sentence in data:
            correct = sentence['test_case'][0]
            typos = sentence['test_case'][1]
            true_correct_label = sentence['label']
            true_typo_label = sentence['label']

            pred_correct = model.predict(correct)
            pred_typo = model.predict(typos)
            
            if len(pred_correct['verbs']) >0: 
                pred_correct_label = pred_correct['verbs'][0]['tags'][1]
            else: 
                pred_correct_label = 0
                
            if len(pred_typo['verbs']) >0:
                pred_typo_label = pred_typo['verbs'][0]['tags'][1]
            else:
                pred_typo_label = 0
            
            total += 2
            
            if pred_correct_label != true_correct_label:
                failures += 1
                failed_sentences.append(["C", correct, true_correct_label, pred_correct_label])
            if pred_typo_label != true_typo_label:
                failures += 1
                failed_sentences.append(["T", typos, true_typo_label, pred_typo_label])

            pred_labels_correct.append(pred_correct_label)
            true_labels_correct.append(true_correct_label)
            pred_labels_typos.append(pred_typo_label)
            true_labels_typos.append(true_typo_label)
        
        percentage = (failures/total)*100
        for sentence in failed_sentences:
            if sentence[0] == "C":
                correct_fails.append(sentence)
            else:
                typos_fails.append(sentence)

        
        print(f"Number of test cases: {total}.")
        print(f"Failures: {failures} ({percentage:.3f}%).\n")
        # Evaluate performance on correct test cases
        if len(correct_fails) > 1:
            print(f"--- Normal sentences test cases ---")
            print(f"Examples of failures")
            for i in range(0,4):
                print(f"Sentence: {correct_fails[i][1]}")
                print(f"True labels: {correct_fails[i][2]}")
                print(f"Predicted labels: {correct_fails[i][3]}\n")

        # Evaluate performance on typo test cases
        if len(typos_fails) > 1:
            print(f"--- Typo sentences test cases ---")
            print(f"Examples of failures")
            for i in range(0,4):
                print(f"Sentence: {typos_fails[i][1]}")
                print(f"True labels: {typos_fails[i][2]}")
                print(f"Predicted labels: {typos_fails[i][3]}\n")
    print("\n")

In [44]:
evaluate_robustness(["BERT", "BiLSTM"], robustness)

Evaluation of capability: robustness, using model: BERT.

Number of test cases: 240.
Failures: 56 (23.333%).

--- Normal sentences test cases ---
Examples of failures
Sentence: The gurl ruuns to mommy.
True labels: I-ARG0
Predicted labels: 0

Sentence: The gurl ruuns to mummy.
True labels: I-ARG0
Predicted labels: 0

Sentence: The gurl ruuns to mommie.
True labels: I-ARG0
Predicted labels: 0

Sentence: The gurl ruuns to momy.
True labels: I-ARG0
Predicted labels: 0

Evaluation of capability: robustness, using model: BiLSTM.

Number of test cases: 240.
Failures: 175 (72.917%).

--- Normal sentences test cases ---
Examples of failures
Sentence: The girl runs to mommy.
True labels: I-ARG0
Predicted labels: I-ARG1

Sentence: The girl runs to mummy.
True labels: I-ARG0
Predicted labels: I-ARG1

Sentence: The girl runs to mommie.
True labels: I-ARG0
Predicted labels: I-ARG1

Sentence: The girl runs to momy.
True labels: I-ARG0
Predicted labels: I-ARG1

--- Typo sentences test cases ---
Examp

## Evaluation capability of Switching Object and Subject

In [45]:
def evaluate_switching(models, data):
    for model_name in models: 
        if model_name == "BERT":
            model = bert
        if model_name == "BiLSTM":
            model = bilstm
            
        print(f"Evaluation of capability: subject-object switching, using model: {model_name}.\n")
            
        pred_labels_normal = []
        true_labels_normal = []
        pred_labels_switched = []
        true_labels_switched = []
        failed_sentences = []
        total, failures = 0, 0
        normal_fails, switched_fails = [], []

        for sentence in switching:
            normal = sentence['test_case'][0]
            switched = sentence['test_case'][1]
            true_normal_label = sentence['label']
            true_switched_label = sentence['label']
            pred_normal = model.predict(normal)
            pred_switched = model.predict(switched)

            if len(pred_normal['verbs']) > 0: 
                pred_normal_label = pred_normal['verbs'][0]['tags']
            else: 
                pred_normal_label = 0

            if len(pred_switched['verbs']) > 0:
                pred_switched_label = pred_switched['verbs'][0]['tags']
            else:
                pred_switched_label = 0

            total += 2

            if pred_normal_label != true_normal_label:
                failures += 1
                failed_sentences.append(["N", normal, true_normal_label, pred_normal_label])
            if pred_switched_label != true_switched_label:
                failures += 1
                failed_sentences.append(["S", switched, true_switched_label, pred_switched_label])

            pred_labels_normal.append(pred_normal_label)
            true_labels_normal.append(true_normal_label)
            pred_labels_switched.append(pred_switched_label)
            true_labels_switched.append(true_switched_label)

        percentage = (failures/total)*100
        for sentence in failed_sentences:
            if sentence[0] == "N":
                normal_fails.append(sentence)
            else:
                switched_fails.append(sentence)

        print(f"Number of test cases: {total}.")
        print(f"Failures: {failures} ({percentage:.3f}%).\n")
        # Evaluate performance on normal test cases
        print(f"--- Normal sentences test cases ---")
        print(f"Examples of failures")
        for i in range(0,4):
            print(f"Sentence: {normal_fails[i][1]}")
            print(f"True labels: {normal_fails[i][2]}")
            print(f"Predicted labels: {normal_fails[i][3]}\n")

        # Evaluate performance on swithced test cases
        print(f"--- Switched sentences test cases ---")
        print(f"Examples of failures")
        for i in range(0,4):
            print(f"Sentence: {switched_fails[i][1]}")
            print(f"True labels: {switched_fails[i][2]}")
            print(f"Predicted labels: {switched_fails[i][3]}\n")
        
    print("\n")

In [46]:
evaluate_switching(['BERT', 'BiLSTM'], switching)

Evaluation of capability: subject-object switching, using model: BERT.

Number of test cases: 160.
Failures: 33 (20.625%).

--- Normal sentences test cases ---
Examples of failures
Sentence: Elise likes Ann.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: ['B-V', 'O', 'O', 'O']

Sentence: Elise hates Ann.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: ['B-V', 'O', 'O', 'O']

Sentence: Frank hugs Ann.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: 0

Sentence: Marie hugs Ann.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: 0

--- Switched sentences test cases ---
Examples of failures
Sentence: Ann hugs Frank.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: 0

Sentence: Ann hugs Marie.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: 0

Sentence: Ann hugs Elise.
True labels: ['B-ARG0', 'B-V', 'B-ARG1', 'O']
Predicted labels: 0

Sentence: Rose hugs Pete.
True labels: ['B-ARG0', 'B-V', 'B-ARG1'

## Evaluation capability of Caused Motion

In [47]:
def evaluate_caused_motion(models, data):
    for model_name in models: 
        if model_name == "BERT":
            model = bert
        if model_name == "BiLSTM":
            model = bilstm
  
        print(f"Evaluation of capability: caused motion, using model: {model_name}.\n")
    
        pred_labels, true_labels = [], []
        failures, total = 0, 0
        fails = []
        
        for sentence in data:
            sent = sentence['test_case']
            true_label = sentence['label'][1]
            pred = model.predict(sent)
            if len(pred['verbs']) > 0:
                pred_label = pred['verbs'][0]['tags'][1]
            else:
                pred_label = 0

            total += 1
    
            if pred_label != true_label:
                fails.append([sent, true_label, pred_label])
                failures += 1

            pred_labels.append(pred_label)
            true_labels.append(true_label)

        percentage = (failures/total)*100
        
        print(f"Number of test cases: {total}.")
        print(f"Failures: {failures} ({percentage:.3f}%).\n")
        # Evaluate performance on passive voice test cases

        print(f"--- Test cases ---")
        print(f"Examples of failures")
        for i in range(0,4):
            print(f"Sentence: {fails[i][0]}")
            print(f"True labels: {fails[i][1]}")
            print(f"Predicted labels: {fails[i][2]}\n")
    print("\n")

In [48]:
evaluate_caused_motion(['BERT', "BiLSTM"], caused_motion)

Evaluation of capability: caused motion, using model: BERT.

Number of test cases: 50.
Failures: 30 (60.000%).

--- Test cases ---
Examples of failures
Sentence: The child dragged.
True labels: I-ARG1
Predicted labels: I-ARG0

Sentence: The hammer struck.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The man pushed.
True labels: I-ARG1
Predicted labels: I-ARG0

Sentence: The girl threw.
True labels: I-ARG1
Predicted labels: I-ARG0

Evaluation of capability: caused motion, using model: BiLSTM.

Number of test cases: 50.
Failures: 17 (34.000%).

--- Test cases ---
Examples of failures
Sentence: The hammer struck.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The man pushed.
True labels: I-ARG1
Predicted labels: I-ARG0

Sentence: The girl threw.
True labels: I-ARG1
Predicted labels: I-ARG0

Sentence: The bulldozer pushed.
True labels: I-ARG1
Predicted labels: I-ARG0





In [49]:
evaluate_voice(["BERT", "BiLSTM"], passive_voice)
evaluate_robustness(["BERT", "BiLSTM"], robustness)
evaluate_switching(['BERT', 'BiLSTM'], switching)
evaluate_caused_motion(['BERT', "BiLSTM"], caused_motion)

Evaluation of capability: passive voice, using model: BERT.

Number of test cases: 384.
Failures: 2 (0.521%).

--- Passive sentences test cases ---
Examples of failures
Sentence: The writer wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The author wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARGM-LOC

Evaluation of capability: passive voice, using model: BiLSTM.

Number of test cases: 384.
Failures: 4 (1.042%).

--- Passive sentences test cases ---
Examples of failures
Sentence: The writer wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The author wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The novelist wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2

Sentence: The creator wrote the journal.
True labels: I-ARG1
Predicted labels: I-ARG2



Evaluation of capability: robustness, using model: BERT.

Number of test cases: 240.
Failures: 56 (23.333%).

--- Normal sentences test